# __ASE SchNet implementation__
This notebook contains the installation procedure and initial testing provided in the SchNetPack [GitHub](https://github.com/atomistic-machine-learning/schnetpack/tree/master) and the [documentation](https://schnetpack.readthedocs.io/en/latest/) for installing SchNet NNP. 

### __Installation procedure__
Refer to ASE installation for the basics of the miniconda `venv` where MACE will be installed. This ensures that other calculators will not break the environment.
- ASE installation process
    - Create new conda environment (python 3.10 is standard). Check the Python requirement for the external calculator wanted. Usually  `python>3.9`.  
    - `conda install ase` (installs `scipy` and `numpy` dependencies) and `conda install matplotlib`. 
    - Check the environment works installing `conda install pytest` and `ase test`.
    - For ASE representations inside Jupyter Notebook, conda install `notebook`, `ipywidgets` and `nglview`. 
- SchNetPack installation process
    - Clone base ASE environment. 
    - Install `pytorch>=1.9` via `conda install pytorch=2.2` (the flag pytorch=2.2 caused problems and had to install 2.5.1) and `pyTorchLightning` `conda install pytorchlightning`. **Note.** `pytorchlightning` was not found...
    - Additional dependencies include `ase>=3.21` and `hydra>1.1.0` via `pip install hydra`. 
    - Visualization is compatible with `tensorboard`, `conda install tensorboard`. 

**Alternative installation.** Just use `pip install --upgrade schnetpack`, takes care of all the packages, but will make the venv completely unflexible. `pip install tensorboard`. 

**Testing.** To test the installation has been successful
- Complete the [tutorials](https://github.com/atomistic-machine-learning/schnetpack/tree/master/examples/tutorials), or 
- Download the `tests` directory from the repo (install it in the conda `site-packages` folder of the venv `venv/x/lib/python3.10/site-packages`). Once propertly installed go to `/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/schnetpack` and run `pytest tests`. Check [here](https://github.com/atomistic-machine-learning/schnetpack/tree/master/tests) for more info. 


**SchNetPack CLI issue.** The CLI is based on Hydra and oriented on pytorch lightning. The template can be found [here](https://github.com/ashleve/lightning-hydra-template).  Note that in order to be able to use the SchNetPack CLI a training script called `spktrain` must be added to `$PATH`. I did not find that file nor was it added to path (the installation was through conda, it cannot escape the venv nor add things to `$PATH`)... 

As such CLI usage will be neglected, and only the ASE interface will be used. 


### __References__

Introductory tutorials for `SchNetPack` (via python/ASE interface) can be found in the [GitHub repo](https://github.com/atomistic-machine-learning/schnetpack/tree/master/examples/tutorials) and in the  [documentation](https://schnetpack.readthedocs.io/en/latest/tutorials/tutorial_01_preparing_data.html). Check the `schnetpack_tutorials` directory for the tutorials. 

## __SchNetPack pretrained model. SchNet__
SchNetPack provides a wide variety of [datasets](https://schnetpack.readthedocs.io/en/latest/api/datasets.html) which can be used to train models on organic molecules (e.g. ANI1 dataset) or on bulk materials (e.g. Materials Project dataset). Similarly to MACE, SchNetPack consists of pretrained models and code to train a model using SchNet architecture on some training dataset. 

Once a model has been trained on some dataset (see QM9 training tutorial), we can use it to evaluate the model and make predictions. Similarly to TorchANI, two separate ways of using the model exists 
- **SchNetPack (SPK) native usage** 
    - The `Trainer` object stores the best model in the model directory which can be loaded using pyTorch. Check [this documentation](https://schnetpack.readthedocs.io/en/latest/tutorials/tutorial_02_qm9.html) (Inference part) for more info. 
    - If the data (structures) is not SchNetPack format, we can define the structures as ASE `Atoms` and `AtomsConverter` from `spk`. 

- **Usage via ASE calculator.** 
    - Alternatively, we can use `SpkCalculator` as an interface to ASE. It requires a path to a trained model (SPK model), a neighbour list, and names and units of the properties used in the model (these consist of the inputs of the trained model). 
    - (kwargs) Precision and the device used can be controlled with `dtype` and `device`. 
    - **Note.** The calculator automatically converts the prediction of the given unit to internal ASE units (eV, Å, ...). 

    The `SpkCalculator` offers a simple way to perform all computations available in the ASE package via a trained model (provides the forcefield). This ASE computations include optimization, NMO and simple MD simulations.   

Additional ASE interface info can be found in the documentation [Interface to ASE](https://schnetpack.readthedocs.io/en/latest/tutorials/tutorial_03_force_models.html#Interface-to-ASE), within tutorial 3.

In [ ]:
## ase imports
from ase import units

# import the calculators
import schnetpack as spk

#=======================#
#       CALCULATORS     #   
#=======================#

calculator = spk.interfaces.SpkCalculator(
    model_file=MODEL_PATH,                          # path to the pretrained model
    neighbor_list=trn.ASENeighborList(cutoff=5.0),  # SchNetPack neighbor list
    energy_key=MD17.energy,                         # name of energies in model
    force_key=MD17.forces,                          # name of forces in model
    energy_unit="kcal/mol",                         # energy unit used by the model
    position_unit="Ang",                            # length unit used by the model
    device='cpu'                                    # device used for calculations
    dtype='float32'                                 # model precision 
)

# the usage is the same as any other ASE calculator

## __1. Training SchNet models__

### __1.1. Training a SchNet model on the QM9 training set.__
References: Tutorial 1 and [Tutorial 2](https://schnetpack.readthedocs.io/en/latest/tutorials/tutorial_02_qm9.html) of the documentation. 

To train a model, first the train dataset must be preprocessed.

In [2]:
import os
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

import torch
import torchmetrics
import pytorch_lightning as pl

qm9tut = './qm9tut'
if not os.path.exists('qm9tut'):
    os.makedirs(qm9tut)

In [3]:
%rm split.npz

qm9data = QM9(
    './qm9.db',
    batch_size=100,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        trn.RemoveOffsets(QM9.U0, remove_mean=True, remove_atomrefs=True),
        trn.CastTo32()
    ],
    property_units={QM9.U0: 'eV'},
    num_workers=1,
    split_file=os.path.join(qm9tut, "split.npz"),
    pin_memory=True,            # set to false, when not using a GPU
    load_properties=[QM9.U0],   #only load U0 property
)
qm9data.prepare_data()
qm9data.setup()

rm: split.npz: No such file or directory


INFO:root:Downloading GDB-9 atom references...
INFO:root:Done.
INFO:root:Downloading GDB-9 data...
INFO:root:Done.
INFO:root:Extracting files...
INFO:root:Done.
INFO:root:Parse xyz files...
100%|██████████| 133885/133885 [04:21<00:00, 511.63it/s]
INFO:root:Write atoms to db...
INFO:root:Done.
100%|██████████| 10/10 [00:27<00:00,  2.74s/it]


Note that neighbors are collected using neighborlists. This is necessary for evaluating new (previously unseen) data for inference via an ASE calculator. Note that a cutoff is imposed to handle large molecules and PBCs (environment truncation, usage of locality). 

In [4]:
# DB info
print('Number of reference calculations:', len(qm9data.dataset))
print('Number of train data:', len(qm9data.train_dataset))
print('Number of validation data:', len(qm9data.val_dataset))
print('Number of test data:', len(qm9data.test_dataset))
print('Available properties:')

for p in qm9data.dataset.available_properties:
    print('-', p)

Number of reference calculations: 133885
Number of train data: 1000
Number of validation data: 1000
Number of test data: 131885
Available properties:
- rotational_constant_A
- rotational_constant_B
- rotational_constant_C
- dipole_moment
- isotropic_polarizability
- homo
- lumo
- gap
- electronic_spatial_extent
- zpve
- energy_U0
- energy_U
- enthalpy_H
- free_energy
- heat_capacity


In [5]:
example = qm9data.dataset[0]
print('Properties:')

for k, v in example.items():
    print('-', k, ':', v.shape)

Properties:
- _idx : torch.Size([1])
- energy_U0 : torch.Size([1])
- _n_atoms : torch.Size([1])
- _atomic_numbers : torch.Size([5])
- _positions : torch.Size([5, 3])
- _cell : torch.Size([1, 3, 3])
- _pbc : torch.Size([3])


In [6]:
atomrefs = qm9data.train_dataset.atomrefs
print('U0 of hyrogen:', atomrefs[QM9.U0][1].item(), 'eV')
print('U0 of carbon:', atomrefs[QM9.U0][6].item(), 'eV')
print('U0 of oxygen:', atomrefs[QM9.U0][8].item(), 'eV')

U0 of hyrogen: -13.613121032714844 eV
U0 of carbon: -1029.863037109375 eV
U0 of oxygen: -2042.611083984375 eV


**Model definition.** Once the data has been preprocessed, we must define the model architecture and the training (representation, optimization, regulation, etc.). The procedure is as follows:
- Input modules to prepared the batched data before building the representation
    - Calculation of pariwise distances

- Representation
    - SchNet (uses atom-wise features)

- Output modules for property prediction

In [7]:
# SchNet, 3 interaction layers, 5 Å coine cutoff, 
# pairwise distances expanded on 20 gaussians and 
# 50 atom-wise features and convolution filters

# representation parameters
cutoff = 5.
n_atom_basis = 30

pairwise_distance = spk.atomistic.PairwiseDistances()
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)

# atom-wise energy contributions are then summed
# to predict the energy using an `Atomwise` module. 
pred_U0 = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=QM9.U0)


# construct the NNP
nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],
    output_modules=[pred_U0],
    postprocessors=[trn.CastTo64(), trn.AddOffsets(QM9.U0, add_mean=True, add_atomrefs=True)]
)


# output modules
output_U0 = spk.task.ModelOutput(
    name=QM9.U0,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1.,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)


# task??
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_U0],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


**Training the model.** Optimizer, callbacks, regulation, etc. 

In [10]:
logger = pl.loggers.TensorBoardLogger(save_dir=qm9tut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(qm9tut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    accelerator='cpu',
    callbacks=callbacks,
    logger=logger,
    default_root_dir=qm9tut,
    max_epochs=3, # for testing, we restrict the number of epochs

)
trainer.fit(task, datamodule=qm9data)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name    | Type                   | Params | Mode 
-----------------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K | train
1 | outputs | ModuleList             | 0      | train
-----------------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)
60        Modules in train mode
0         Modules in eval mode
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'outputs' parameter because it is not possible to safely dump to YAML.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


**Inference.** Once the model has been trained with the data, we can use the model for inference. The model is saved as `best_inference_model` when trained. A SpkCalculator can be defined with this model. This calculator interface makes sense if we want to predict the PES of a system. 

In [12]:
qm9calculator = spk.interfaces.SpkCalculator(
    model_file=os.path.join(qm9tut, "best_inference_model"),    # path to model
    neighbor_list=trn.ASENeighborList(cutoff=5.),               # neighbor list
    energy_key=QM9.U0,          # name of energy property in model
    energy_unit="eV",           # units of energy property
    device='cpu',               # device for computation
)

INFO:schnetpack.interfaces.ase_interface:Loading model from ./qm9tut/best_inference_model
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/schnetpack/utils/compatibility.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

### __1.2. Training a model on forces and energies (from an AIMD).__
In this [tutorial](https://schnetpack.readthedocs.io/en/latest/tutorials/tutorial_04_molecular_dynamics.html), a SchNet model is trained on energies and forces using the MD17 ethanol dataset as an example. Then, the model performance is evaluated and geometry optimisation, normal mode analysis and basic molecular dynamic simulations are performed using the SchNetPack ASE interface. 

In [16]:
import torch
import torchmetrics
import schnetpack as spk
import schnetpack.transform as trn
import pytorch_lightning as pl
import os
import matplotlib.pyplot as plt
import numpy as np

forcetut = './forcetut'
if not os.path.exists(forcetut):
    os.makedirs(forcetut)

As a first step, the MD17 ethanol dataset is loaded (same protocol as in QM9 tutorial).

In [17]:
from schnetpack.datasets import MD17

ethanol_data = MD17(
    os.path.join(forcetut,'ethanol.db'),
    molecule='ethanol',
    batch_size=10,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        trn.RemoveOffsets(MD17.energy, remove_mean=True, remove_atomrefs=False),
        trn.CastTo32()
    ],
    num_workers=1,
    pin_memory=False, # set to false, when not using a GPU
)
ethanol_data.prepare_data()
ethanol_data.setup()

INFO:root:Downloading ethanol data
INFO:root:Parsing molecule ethanol
INFO:root:Write atoms to db...
INFO:root:Done.
100%|██████████| 100/100 [00:09<00:00, 10.98it/s]


In [18]:
# dataset info
properties = ethanol_data.dataset[0]
print('Loaded properties:\n', *['{:s}\n'.format(i) for i in properties.keys()])

print('Forces:\n', properties[MD17.forces])
print('Shape:\n', properties[MD17.forces].shape)

Loaded properties:
 _idx
 energy
 forces
 _n_atoms
 _atomic_numbers
 _positions
 _cell
 _pbc

Forces:
 tensor([[ 1.4517e+00,  6.0192e+00,  5.2068e-07],
        [ 1.7953e+01, -5.1624e+00,  3.4900e-07],
        [-4.0884e+00,  2.2590e+01,  3.3088e-06],
        [-1.1416e+00, -9.7469e+00,  7.6473e+00],
        [-1.1416e+00, -9.7469e+00, -7.6473e+00],
        [-2.4821e+00,  4.9335e+00,  4.3700e+00],
        [-2.4821e+00,  4.9335e+00, -4.3700e+00],
        [-5.5148e+00, -3.0207e+00, -8.9093e-09],
        [-2.4393e+00, -1.0838e+01, -6.0721e-08]], dtype=torch.float64)
Shape:
 torch.Size([9, 3])


Once the dataset has been defined, we can define the model. As before, we must define the input modules, build the representation and define the output modules. Compared to the QM9 tutorial, this time we want to model forces (derivative of the energy). 

In [25]:
cutoff = 5.
n_atom_basis = 30

# input
pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)

# representation
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)

# output
pred_energy = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=MD17.energy)
pred_forces = spk.atomistic.Forces(energy_key=MD17.energy, force_key=MD17.forces)


nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],
    output_modules=[pred_energy, pred_forces],
    postprocessors=[
        trn.CastTo64(),
        trn.AddOffsets(MD17.energy, add_mean=True, add_atomrefs=False)
    ]
)

Once the model has been defined, we can train the model. The loss function is comprised of a significant part force loss and another energy loss. 

In [26]:
# loss function
output_energy = spk.task.ModelOutput(
    name=MD17.energy,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.01,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

output_forces = spk.task.ModelOutput(
    name=MD17.forces,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.99,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

# train task
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_energy, output_forces],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [28]:
# model training
logger = pl.loggers.TensorBoardLogger(save_dir=forcetut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(forcetut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    accelerator='cpu',
    callbacks=callbacks,
    logger=logger,
    default_root_dir=forcetut,
    max_epochs=5, # for testing, we restrict the number of epochs
)
trainer.fit(task, datamodule=ethanol_data)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type                   | Params | Mode 
-----------------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K | train
1 | outputs | ModuleList             | 0      | train
-----------------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'outputs' parameter because it is not possible to safely dump to YAML.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


With the model trained, we can cast it to an ASE calculator

In [33]:
ethanol_calc = spk.interfaces.SpkCalculator(
    model_file=os.path.join(forcetut, "best_inference_model"),  # path to model
    neighbor_list=trn.ASENeighborList(cutoff=5.0),              # neighbor list
    energy_key=MD17.energy,         # name of energy property in model
    force_key=MD17.forces,          # name of force property in model
    energy_unit="kcal/mol",         # unit of energy property
    position_unit="Ang",            # unit of length property
    device='cpu',                   # device for computation 
)

INFO:schnetpack.interfaces.ase_interface:Loading model from ./forcetut/best_inference_model
/Users/sergiortizropero/miniconda3/envs/ASE_schnet/lib/python3.10/site-packages/schnetpack/utils/compatibility.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Ple

## __2. ASE SchNet calculator usage__
Once a model has been trained on some dataset (see `qm9calculator` from the QM9 tutorial and the `md17calculator` from the ethanol dataset tutorial), we avaluate the model to make predictions.


### __EtOH single point calculation and atomization energy__

In [38]:
from ase import build
atoms = build.molecule('CH3CH2OH')

atoms.calc = qm9calculator
qm9_etOH_energy = atoms.get_potential_energy()
print(f'EtOH energy @ SchNet-QM9:\t{qm9_etOH_energy:.3f} kcal/mol')

atoms.calc = ethanol_calc
md17_etOH_energy = atoms.get_potential_energy()
print(f'EtOH energy @ SchNet-MD17:\t{md17_etOH_energy:.3f} kcal/mol')

EtOH energy @ SchNet-QM9:	-4221.675 kcal/mol
EtOH energy @ SchNet-MD17:	-4215.754 kcal/mol


In [40]:
# check the energy of formation of the H2O molecule
calc_list = [qm9calculator, ethanol_calc]
atomic_energies = []

k = 0
for calc_ in calc_list:
    # define the atoms
    atom_O = build.molecule('O')
    atom_C = build.molecule('C')
    atom_H = build.molecule('H')

    atom_O.calc = calc_
    atom_C.calc = calc_
    atom_H.calc = calc_

    energy_O = atom_O.get_potential_energy()
    energy_C = atom_C.get_potential_energy()
    energy_H = atom_H.get_potential_energy()
    
    atomic_energies.append([energy_O, energy_C, energy_H])

    k += 0

# check the results
delta_E_formation_qm9 = qm9_etOH_energy - (atomic_energies[0][0] + 2 * atomic_energies[0][1] + 6 * atomic_energies[0][2])
delta_E_formation_md17 = md17_etOH_energy - (atomic_energies[0][0] + 2 * atomic_energies[0][1] + 6 * atomic_energies[0][2])

print(atomic_energies)
print(f'EtOH energy formation @ QM9-SchNet:\t{delta_E_formation_qm9:.3f} [kcal/mol]')
print(f'EtOH energy formation @ MD17-SchNet:\t{delta_E_formation_md17:.3f} [kcal/mol]')

[[-2047.2430908083916, -1032.4109637737274, -18.46676814556122], [-468.2428506141119, -468.29980287254796, -468.2469250298014]]
EtOH energy formation @ QM9-SchNet:	1.191 [kcal/mol]
EtOH energy formation @ MD17-SchNet:	7.111 [kcal/mol]
